In [2]:
from __future__ import print_function
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # warning level
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # use GPU 0
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import sys
from math import ceil


In [3]:
import pandas as pd

In [4]:
def loadTrainVars(ID):
    global num_batches_per_epoch
    global trainX1
    global trainX2
    global trainYcat
    global n_input1
    global n_input2
    global trainX
    # load train variables
    vars_path_ID = vars_path + ID + '/'
    trainX = np.load(vars_path_ID+'trainX.npy')
    trainYcat = np.load(vars_path_ID+'trainYcat.npy')
    #print(trainX)
    #df = pd.DataFrame(trainX)
    #df

    # mask some parts to make the desired signal
    mask1 = np.zeros((780,), dtype=bool)
    mask2 = np.zeros((780,), dtype=bool)
    mask1[0:4]=1
    mask1[0+390:4+390]=1
    mask1[4:4+250+1]=1
    mask1[4+390:4+390+250+1]=1
    mask2[0:4]=1
    mask2[0+390:4+390]=1
    mask2[4+250+1:4+250+1+135]=1
    mask2[4+250]=1
    mask2[4+390+250+1:4+390+250+1+135]=1
    mask2[4+390+250]=1
    trainX1 = trainX[:,:,mask1]
    trainX2 = trainX[:,:,mask2]

    # make some general numbers
    num_examples = trainX.shape[0]
    num_batches_per_epoch = ceil(num_examples/batch_size)
    #print('trX1->',trainX1.shape)
    #print('trX2->',trainX2.shape)
    n_input1 = (trainX1.shape[2]//n_steps1)

    n_input2 = (trainX2.shape[2]//n_steps2)
    trainX1 = trainX1.reshape((trainX1.shape[0], n_steps1, n_input1))
    trainX2 = trainX2.reshape((trainX2.shape[0], n_steps2, n_input2))

    return


In [9]:
vars_path = '../Data/AA/'
models_path = '../models/'
logs_path = './Logs/'
n_classes = 7
n_steps = 10
n_hidden = 30
num_epochs = 100
num_layers = 1
batch_size = 100
learning_rate = 1e-3
ret = 0
n_steps1 = n_steps
n_steps2 = n_steps
n_hidden1 = n_hidden
n_hidden2 = n_hidden
#IDs = ['100','101','103','105','106','108','109','111','112','113','114','115','116','117','118','119','121','122','123','124','200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234'] # all records
IDs = ['100','101','103','105','106','108','109','200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215'] # all records
#IDs = ['200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']
#runs = np.random.permutation(np.arange(int(sys.argv[1]), int(sys.argv[2])))#ret#['_1', '_2', '_3', ..., '_50']
runs = np.random.permutation(np.arange(int(1), int(10)))

This is architecture of model-alpha. x1 and x2 are passed to separate LSTM unit and their output is cascaded and passed to FC neural network which will give final prediction.
<img src="modela.gif" , width=100, height=200>


In [6]:
def buildGraph():
    global x1
    global x2
    global y
    global weights
    global biases
    global cost
    global optimizer
    global correct_pred
    global accuracy
    global pred
    global init
    global saver

    with tf.device('/gpu:0'):

        # reset Graph
        tf.reset_default_graph()

        # tf Graph input
        x1 = tf.placeholder("float", [None, n_steps1, n_input1], name='x1')
        x2 = tf.placeholder("float", [None, n_steps2, n_input2], name='x2')
        y = tf.placeholder("float", [None, n_classes], name='y')

        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
        # Permuting batch_size and n_steps
        x11 = tf.transpose(x1, [1, 0, 2])
        x22 = tf.transpose(x2, [1, 0, 2])
        # Reshaping to (n_steps*batch_size, n_input)
        x11 = tf.reshape(x11, [-1, n_input1])
        x22 = tf.reshape(x22, [-1, n_input2])
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        x11 = tf.split(x11, n_steps1, 0)
        x22 = tf.split(x22, n_steps2, 0)

        # Define weights
        weights = tf.Variable(tf.random_normal([n_hidden1+n_hidden2, n_classes]))
        biases = tf.Variable(tf.random_normal([n_classes]))

        # Define lstm cells with tensorflow
        lstm_cell1 = rnn.LSTMCell(n_hidden1, use_peepholes=False)
        lstm_cell2 = rnn.LSTMCell(n_hidden2, use_peepholes=False)

        # Get outputs of lstm cells
        with tf.variable_scope('rnn1'):
            outputs1, states1 = rnn.static_rnn(lstm_cell1, x11, dtype=tf.float32)
        with tf.variable_scope('rnn2'):
            outputs2, states2 = rnn.static_rnn(lstm_cell2, x22, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        tmp1 = tf.concat(values=[outputs1[-1], outputs2[-1]], axis=1)
        tmp1 = tf.matmul(tmp1, weights)
        pred = tf.add(tmp1, biases, name='pred')

        # Define loss and optimizer
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

        # Evaluate model
        correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='Accuracy')

        # Initializing the variables
        init = tf.global_variables_initializer()

        # write the model
        #writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
#         with tf.Session() as sess:
#             writer = tf.summary.FileWriter("output_graph", sess.graph)
#             print(sess.run(pred))
#             writer.close()

        # 'Saver' op to save and restore all the variables
        saver = tf.train.Saver()

    return


In [7]:
def runModel(ID):
    global sess
    global predY
    global trainX1
    global trainX2
    global trainYcat

    # set configs
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True # usw memory of GPUs more efficiently

    # Launch the graph
    with tf.Session(config=config) as sess:
        sess.run(init)

        for curr_epoch in range(num_epochs):
            for batch in range(num_batches_per_epoch):

                # Preparing required batch
                batchRange = range(batch*batch_size, min((batch+1)*batch_size,trainX1.shape[0]))
                batch_x1 = trainX1[batchRange,:,:]
                batch_x2 = trainX2[batchRange,:,:]
                batch_y = trainYcat[batchRange,:]

                # Reshape data to get seq of required elements
                batch_x1 = batch_x1.reshape((batch_x1.shape[0], n_steps1, n_input1))
                batch_x2 = batch_x2.reshape((batch_x2.shape[0], n_steps2, n_input2))

                # Run optimization op (backprop)
                sess.run(optimizer, feed_dict={x1: batch_x1, y: batch_y, x2:batch_x2})

                # Calculate accuracy of last epoch batch and batch loss
                acc, loss = sess.run([accuracy, cost], feed_dict={x1: batch_x1, y: batch_y, x2:batch_x2})

            #losses.append(loss)
            #acces.append(acc)

        # save the prediction of train datas
        predY = sess.run(pred, feed_dict={x1: trainX1, y: trainYcat, x2:trainX2})
        directory = os.path.dirname(predsPath)
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
        np.save( predsPath + ID + '_predY.npy', predY)

        # Save model weights to disk
        directory = os.path.dirname(model_path)
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
        model_path_ID = model_path + ID + '/'
        directory = os.path.dirname(model_path_ID)
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
        del directory
        save_path = saver.save(sess, model_path_ID+'model.ckpt')
        print("ModelA of %s was saved" % str(ID+'_'+str(run)))

    return


In [10]:
for ID in IDs:
    loadTrainVars(ID) #load variables

    for run in runs:
        # path of saving models and their results
        model_path = models_path + 'modelsA_' + str(run) + '_' + str(ret) + '/'
        predsPath = '../preds/trainA_outs_' + str(run) + '_' + str(ret) + '/'
        buildGraph() #make the graph
        runModel(ID) #train the model with train data


ModelA of 100_2 was saved
ModelA of 100_7 was saved
ModelA of 100_5 was saved
ModelA of 100_9 was saved
ModelA of 100_6 was saved
ModelA of 100_4 was saved
ModelA of 100_3 was saved
ModelA of 100_1 was saved
ModelA of 100_8 was saved
ModelA of 101_2 was saved
ModelA of 101_7 was saved
ModelA of 101_5 was saved
ModelA of 101_9 was saved
ModelA of 101_6 was saved
ModelA of 101_4 was saved
ModelA of 101_3 was saved
ModelA of 101_1 was saved
ModelA of 101_8 was saved
ModelA of 103_2 was saved
ModelA of 103_7 was saved
ModelA of 103_5 was saved
ModelA of 103_9 was saved
ModelA of 103_6 was saved
ModelA of 103_4 was saved
ModelA of 103_3 was saved
ModelA of 103_1 was saved
ModelA of 103_8 was saved
ModelA of 105_2 was saved
ModelA of 105_7 was saved
ModelA of 105_5 was saved
ModelA of 105_9 was saved
ModelA of 105_6 was saved
ModelA of 105_4 was saved
ModelA of 105_3 was saved
ModelA of 105_1 was saved
ModelA of 105_8 was saved
ModelA of 106_2 was saved
ModelA of 106_7 was saved
ModelA of 10